# A single compartment SEIR simulator

In [1]:
import numpy as np;
import pandas as pd;
import plotly.express as px;
import plotly.graph_objects as go;

## Simulation Parameters

In [2]:
nDays    = 36    # No.of days to simulate
N        = 3.5 * (10** 7) # Total population
AR0      = 105     # Initial number of reported active cases
# I0       = 1000

## Basic Model Parameters

In [34]:
probTrans   = 0.015
ch          = 5    # Mean household contacts per day
cw          = 15   # Mean workplace contacts per day
eMit        = 1/3    # Total effect on mitigation
tE          = 3
tI          = 7
tP          = 2
tA          = 13
alpha       = 0.1 # Fraction of active cases reported
compSEIRPAC = {"S":0, "E":1, "I":2, "R":3, "P":4, "A":5, "C":6}

## Derived Model Parameters

In [35]:
beta  = probTrans * (ch + cw * eMit)
# rates
rE = 1/tE
rI = 1/tI
rP = 1/tP
rA = 1/tA
R0 = beta/rI
print("R0 = ", R0, "beta = ", beta)

R0 =  1.05 beta =  0.15


## Explicit Estimator for I in in the initial phase

$\lambda_1$ is the largest eigen value of the linearised Jacobian matrix

$$
J = 
\begin{bmatrix}
-r_E & \beta & 0 &  0 & 0 & 0 \\
r_E & -r_I & 0 & 0 & 0 & 0 \\
0 & r_I & 0 & 0 & 0 & 0 \\
r_E & 0 & 0 & -r_P & 0 & 0 \\
0 & 0 & 0 & r_P & -r_A & 0 \\
0 & 0 & 0 & 0 & r_A & 0 \\
\end{bmatrix}
$$

One can also directly find $\lambda_1$ using
$$
\lambda_1 = \frac{-(r_E + r_I) + \sqrt{(r_E- r_I)^2 + 4 r_E \beta}}{2}
$$
Notice that if we assume $S \approx N$ (initial phase of the epidemic spread) then $\dot C = J \times C$,
where $C = \left[E, I, R, P, A, C \right]^T$  

### Eigen analysis of J matrix

In [36]:
# betaS0byN = beta * (N - 8*AR0/alpha)/N
Jmatrix = np.array([
    [-rE, beta, 0,   0,   0, 0],
    [ rE,  -rI, 0,   0,   0, 0],
    [  0,   rI, 0,   0,   0, 0],
    [ rE,    0, 0, -rP,   0, 0],
    [  0,    0, 0,  rP, -rA, 0],
    [  0,    0, 0,   0,  rA, 0],   
])
eigVals, eigVecs = np.linalg.eig(Jmatrix) 
topEigVec = eigVecs[:, np.argmax(eigVals)]
lambda1 = np.max(eigVals)

print("Eigen values :", eigVals)
print("Eigen vectors :\n", eigVecs)

Eigen values : [ 0.          0.         -0.07692308 -0.5        -0.48113905  0.00494857]
Eigen vectors :
 [[ 0.          0.          0.          0.         -0.03525585 -0.01105084]
 [ 0.          0.          0.          0.          0.03474011 -0.024922  ]
 [ 1.          0.          0.          0.         -0.01031484 -0.71945674]
 [ 0.          0.          0.          0.64153303 -0.62308377 -0.00729503]
 [ 0.          0.          0.70710678 -0.7581754   0.77073125 -0.04455161]
 [ 0.          1.         -0.70710678  0.11664237 -0.12322221 -0.6925321 ]]


In [37]:
doublingRate = np.log(2) / lambda1
doublingRate
R0check = (lambda1 + rE)*(lambda1 + rI)/(rE * rI)
lambda1check = (-(rE + rI) + np.sqrt((rE-rI)**2 + 4*rE*beta))/2

(lambda1check, lambda1, R0check, R0, doublingRate)

(0.004948574384282756,
 0.004948574384282783,
 1.0500000000000003,
 1.05,
 140.07007407253633)

### Initial Seeding based on given AR0

In [38]:
A0 = AR0 / alpha # Estimated number of active cases
scaleFromA = topEigVec/topEigVec[4]
initEIRPAC = A0 * scaleFromA
S0 = N - initEIRPAC[0:3].sum()

initSEIRPAC = np.concatenate((np.array([S0]), initEIRPAC))
print("Initial population in each compartment: ", initSEIRPAC)
print("Initial population in E + I + R :", initSEIRPAC[1:4].sum())
print("Initial population in E + P + A + C :", initSEIRPAC[4:7].sum() + initSEIRPAC[1])

Initial population in each compartment:  [3.49821959e+07 2.60448134e+02 5.87365944e+02 1.69562815e+04
 1.71930468e+02 1.05000000e+03 1.63217170e+04]
Initial population in E + I + R : 17804.09562483301
Initial population in E + P + A + C : 17804.095624833008


### Initial Seeding based on given I0

scaleFromI = topEigVec/topEigVec[1]
initEIRPAC = I0 * scaleFromI
S0 = N - initEIRPAC[0:3].sum()

initSEIRPAC = np.concatenate((np.array([S0]), initEIRPAC))
print("Initial population in each compartment: ", initSEIRPAC)
initSEIRPAC[2:4].sum(), initSEIRPAC[4:7].sum(), initSEIRPAC[0:4].sum()

### Closed form Estimation for initial phase

In [39]:
t = np.arange(nDays)
I0 = initSEIRPAC[compSEIRPAC["I"]]
A0 = initSEIRPAC[compSEIRPAC["A"]]

estI = I0 * (np.e ** (lambda1 * t))
estA = A0 * (np.e ** (lambda1 * t))

(I0, A0)

(587.365943629132, 1050.0)

## Simulation

In [40]:
# Initiaize the compartments

tsSEIRPAC = np.zeros((nDays, len(initSEIRPAC))) # Array to store the full time series
tsSEIRPAC[0] = initSEIRPAC

# Run the simulation for nDays
for day in range(1,nDays):
    (S, E, I, R, P, A, C) = tsSEIRPAC[day-1]
    S +=  -beta*S*I/N
    E +=   beta*S*I/N  - E/tE
    I +=   E/tE - I/tI
    R +=   I/tI
    P +=   E/tE - P/tP
    A +=   P/tP - A/tA
    C +=   A/tA
    tsSEIRPAC[day] = (S, E, I, R, P, A, C)

## Time series to Data Frame

In [41]:
dfSEIRPAC = pd.DataFrame(data=tsSEIRPAC, columns=compSEIRPAC)
dfSEIRPAC["estI"] = estI
dfSEIRPAC["estA"] = estA
dfSEIRPAC[0::10]

,S,E,I,R,P,A,C,estI,estA
0,3.498220e+07,260.448134,587.365944,16956.281547,171.930468,1050.000000,16321.717023,587.365944,1050.000000
10,3.498129e+07,274.580070,620.487312,17821.312781,182.067432,1110.048543,17154.640094,617.163378,1103.267143
20,3.498034e+07,289.917463,655.162348,18734.748454,192.236400,1172.725280,18034.857294,648.472454,1159.236562
30,3.497933e+07,306.103078,691.752060,19699.211984,202.969305,1238.523328,18964.580322,681.369859,1218.045343


## Plotting

In [42]:
figAll = go.Figure()
figAll.add_scatter(y = dfSEIRPAC["S"], name="Susceptible (Simulation)")
figAll.add_scatter(y = dfSEIRPAC["E"], name="Exposed (Simulation)")
figAll.add_scatter(y = dfSEIRPAC["I"], name="Infected (Simulation)")
figAll.add_scatter(y = dfSEIRPAC["R"], name="Removed (Simulation)")
figAll.add_scatter(y = dfSEIRPAC["P"], name="Presymptomatic (Simulation)")
figAll.add_scatter(y = dfSEIRPAC["A"], name="Active (Simulation)")
figAll.add_scatter(y = dfSEIRPAC["C"], name="Cured (Simulation)")

In [43]:
figSum = go.Figure()
figSum.add_scatter(y = dfSEIRPAC["I"] + dfSEIRPAC["R"], name="I + R (Simulation)")
figSum.add_scatter(y = dfSEIRPAC["P"] + dfSEIRPAC["A"] + dfSEIRPAC["C"], name="P + A + C (Simulation)")
figSum.update_layout(yaxis_type="log")
figSum.show()

In [44]:
figI = go.Figure()
figI.add_scatter(y = dfSEIRPAC["I"][0:50], name="Infected (Simulation)")
figI.add_scatter(y = dfSEIRPAC["estI"][0:50], name="Infected (Estimate)")
#  figI.add_scatter(y = dfSEIRPAC["S"]/(10**3), name="Susceptible (Simulation)")
figI.show()
figI.update_layout(yaxis_type="log")
figI.show()

In [45]:
figAR = go.Figure()
figAR.add_scatter(y = dfSEIRPAC["A"][0:50]*alpha, name="Active Reported (Simulation)")
figAR.add_scatter(y = dfSEIRPAC["estA"][0:50]*alpha, name="Active Reported (Estimate)")

figAR.show()
figAR.update_layout(yaxis_type="log")
figAR.show()

In [46]:
figEstQ = go.Figure()

figEstQ.add_scatter(y = dfSEIRPAC["estI"]/dfSEIRPAC["I"], name="estI/I")
figEstQ.add_scatter(y = dfSEIRPAC["estA"]/dfSEIRPAC["A"], name="estA/A")
figEstQ.show()

In [47]:
figScale = go.Figure()
figScale.add_scatter(y = dfSEIRPAC["E"]/dfSEIRPAC["A"], name="Exposed/Active")
figScale.add_scatter(y = dfSEIRPAC["I"]/dfSEIRPAC["A"], name="Infected/Active")
#figScale.add_scatter(y = dfSEIRPAC["R"]/dfSEIRPAC["A"], name="Removed/Active")
figScale.add_scatter(y = dfSEIRPAC["P"]/dfSEIRPAC["A"], name="Presymptomatic/Active")
#figScale.add_scatter(y = dfSEIRPAC["C"]/dfSEIRPAC["A"], name="Cured/Active")
figScale.show()

In [48]:
Asim = tsSEIRPAC[:,compSEIRPAC["A"]].reshape(nDays,1)
tsEIRPACbyA = tsSEIRPAC[:,1:] / Asim
scaleFromAobs = tsEIRPACbyA[10:60].mean(axis=0)

print("Scale from A used     : ", scaleFromA)
print("Scale from A obsserved: ", scaleFromAobs)

Scale from A used     :  [ 0.24804584  0.55939614 16.14883957  0.1637433   1.         15.5444924 ]
Scale from A obsserved:  [ 0.247213    0.55866021 15.95948845  0.16392049  1.         15.36357157]
